In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import utils
from tensorflow import config as tfconfig
import pandas as pd
import numpy as np
from random import random
gpus = tfconfig.list_physical_devices('GPU')

if gpus:
    try:
        for gpu in gpus:
            tfconfig.set_logical_device_configuration(gpu, [tfconfig.LogicalDeviceConfiguration(memory_limit=4096)])
    except RuntimeError as e:
        
        print(e)

In [2]:
train = pd.read_csv('train.csv')

In [3]:
train['v'] = 10272*[1]

In [4]:
train = train.pivot(columns='course_id', index='user_id', values='v')

In [5]:
train = train.fillna(0)

In [6]:
train

course_id,1,3,5,6,7,8,9,10,12,13,...,225,227,228,229,234,235,237,238,246,252
user_id,,,,,,,,,,,,,,,,,,,,,
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
186668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
186835,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [8]:
y_train = np.array(train)

In [10]:
y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
model = Sequential()
model.add(Dropout(0.9))
model.add(Dense(512))
model.add(Dense(512))
model.add(Dense(171, activation='sigmoid'))
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

In [13]:
model_save_path = 'best_model.h5'
checkpoint_callback = ModelCheckpoint(model_save_path, 
                                      monitor='val_accuracy',
                                      save_best_only=True,
                                      verbose=1)

history = model.fit(y_train, 
                              y_train, 
                              epochs=60,
                              batch_size=256,
                              validation_split=0.2,
                              callbacks=[checkpoint_callback])

Epoch 1/60


ValueError: in user code:

    File "c:\users\ximal\appdata\local\programs\python\python38\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\users\ximal\appdata\local\programs\python\python38\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\users\ximal\appdata\local\programs\python\python38\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\users\ximal\appdata\local\programs\python\python38\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\users\ximal\appdata\local\programs\python\python38\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\users\ximal\appd

In [474]:
model.load_weights(model_save_path)
sample = pd.read_csv('sample_submission.csv')

In [475]:
np.array(train.loc[120])

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])

In [476]:
users = np.array(sample['user_id'])

In [477]:
x_test = []
for i in users:
    if i in train[1]:
        x_test.append(np.array(train.loc[i]))
    else:
        x_test.append(np.zeros(171))
x_test = np.array(x_test)
x_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [478]:
pred = model.predict(x_test)

91/91 [==============================] - 0s 3ms/step


In [479]:
diff = (pred-x_test)

In [480]:
a1 = np.argmax(diff, axis = 1)

In [481]:
for i in range(len(diff)):
    diff[i][a1[i]] = 0

In [482]:
a2 = np.argmax(diff, axis = 1)

In [483]:
for i in range(len(diff)):
    diff[i][a2[i]] = 0

In [484]:
a3 = np.argmax(diff, axis = 1)

In [485]:
a1

array([2, 2, 2, ..., 2, 2, 2], dtype=int64)

In [486]:
a2

array([75, 75, 30, ...,  0,  0,  0], dtype=int64)

In [487]:
a3

array([ 5, 52, 75, ..., 16, 16, 16], dtype=int64)

In [488]:
for i in range(len(diff)):
    a1[i] = train.columns[a1[i]]
    a2[i] = train.columns[a2[i]]
    a3[i] = train.columns[a3[i]]

In [489]:
a1

array([5, 5, 5, ..., 5, 5, 5], dtype=int64)

In [490]:
a2

array([114, 114,  34, ...,   1,   1,   1], dtype=int64)

In [491]:
a3

array([  8,  85, 114, ...,  20,  20,  20], dtype=int64)

In [492]:
ans = pd.DataFrame(users)[0]
ans1 = pd.DataFrame(a1)[0]
ans2 = pd.DataFrame(a2)[0]
ans3 = pd.DataFrame(a3)[0]

In [493]:
ans

0           78
1           81
2          120
3          123
4          150
         ...  
2895    185864
2896    186262
2897    186691
2898    186887
2899    187206
Name: 0, Length: 2900, dtype: int64

In [494]:
ans1

0       5
1       5
2       5
3       5
4       5
       ..
2895    5
2896    5
2897    5
2898    5
2899    5
Name: 0, Length: 2900, dtype: int64

In [495]:
ans2

0       114
1       114
2        34
3         1
4         1
       ... 
2895      1
2896      1
2897      1
2898      1
2899      1
Name: 0, Length: 2900, dtype: int64

In [496]:
ans3

0         8
1        85
2       114
3        20
4        20
       ... 
2895     20
2896     20
2897     20
2898     20
2899     20
Name: 0, Length: 2900, dtype: int64

In [497]:
df = pd.concat([ans, ans1, ans2, ans3], axis=1)

In [498]:
df = df.set_axis(['user_id','course_id_1','course_id_2','course_id_3'], axis=1)

In [499]:
df

,user_id,course_id_1,course_id_2,course_id_3
0,78,5,114,8
1,81,5,114,85
2,120,5,34,114
3,123,5,1,20
4,150,5,1,20
...,...,...,...,...
2895,185864,5,1,20
2896,186262,5,1,20
2897,186691,5,1,20
2898,186887,5,1,20


In [500]:
df.to_csv('ans.csv')